<a href="https://colab.research.google.com/github/picklerick5/CTE-Intro-To-Machine-Learning/blob/master/digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.utils.np_utils import to_categorical


In [0]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [0]:
train.head()

In [0]:
test.head()

In [0]:
xtr= (train.iloc[:,1:].values).astype('float32') 
ytr = train.iloc[:,0].values.astype('int32')
xte = test.values.astype('float32')

In [0]:
xtr=xtr.reshape(xtr.shape[0],28,28)
xte=xte.reshape(xte.shape[0],28,28)

In [0]:
plt.subplot(337)
plt.imshow(xtr[6], cmap=plt.get_cmap('gray'))
plt.title= ytr[6]

In [0]:
xtr=xtr.reshape(xtr.shape[0],28,28,1)
xte=xte.reshape(xte.shape[0],28,28,1)

In [0]:
meanpx=xtr.mean().astype(np.float32)
stdpx=xtr.std().astype(np.float32)
def standardize(x): 
    return (x-meanpx)/stdpx

In [0]:
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D

In [0]:
model= Sequential()
model.add(Lambda(standardize, input_shape=(28,28,1)))
model.add(Convolution2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

In [0]:
 model.compile(Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
 model.optimizer.lr=0.01
 ytr=to_categorical(ytr)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xtr, ytr, test_size=0.33)

In [0]:
from keras.preprocessing import image
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3, height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(x_train, y_train, batch_size=64)
test_batches = gen.flow(x_test, y_test, batch_size=64)
model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, validation_data=test_batches, validation_steps=test_batches.n)

In [0]:
batches = gen.flow(xtr, ytr, batch_size=64)
model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=3)

In [0]:
predictions = model.predict_classes(xte, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("Digits.csv", index=False, header=True)